## Model Monitor Pipeline

In [849]:
#  import sys
#  !{sys.executable} -m pip install --upgrade pip
#  !{sys.executable} -m pip install -qU awscli boto3 "sagemaker>=2.0.0"
#  !{sys.executable} -m pip install -qU "stepfunctions>=2.0.0"
#  !{sys.executable} -m pip show sagemaker stepfunctions

In [850]:
import pandas as pd
from stepfunctions.inputs import ExecutionInput


In [851]:
import sagemaker
import stepfunctions
import logging
from stepfunctions.steps import *
from stepfunctions.workflow import Workflow
from stepfunctions import steps
from stepfunctions.inputs import ExecutionInput
from sagemaker.processing import Processor,ProcessingInput, ProcessingOutput
import uuid
import sagemaker
from sagemaker.inputs import TrainingInput
import boto3
from sagemaker.network import NetworkConfig
from sagemaker.network import NetworkConfig
stepfunctions.set_stream_logger(level=logging.INFO)




### Parameter definitions

In [852]:
#added on March 30
import datetime
year=datetime.datetime.now().strftime("%Y")
month=datetime.datetime.now().strftime("%m")
day=datetime.datetime.now().strftime("%d")
hour=datetime.datetime.now().strftime("%H")
ehour=datetime.datetime(int(year),int(month),int(day),int(hour)+1).strftime("%H")
timestr=("{}/{}/{}/{}".format(year,month,day,hour))
ep_strttime=("{}-{}-{}T{}:{}:{}Z".format(year,month,day,hour,"00","00"))
ep_endtime=("{}-{}-{}T{}:{}:{}Z".format(year,month,day,ehour,"00","00"))
print(timestr)
print(ep_strttime)
print(ep_endtime)


2022/04/04/21
2022-04-04T21:00:00Z
2022-04-04T22:00:00Z


In [853]:
#updated few parameters here-
#change only below one for VW env details

v_workflow_execution_role = "arn:aws:iam::014257795134:role/ds-mlops-stepfunction-role" # Step function IAM role ARN
v_preprocessing_iam_role = "arn:aws:iam::014257795134:role/ds-mlops-sagemaker-role" # IAM role for preprocessing container
#make sure to add SNS Full acess to below role to enable SNS notification.
v_lambda_execution_role = "arn:aws:iam::014257795134:role/role_lambda_mlops" # SNS

v_glue_role='arn:aws:iam::014257795134:role/Glue_Role_MLOPS'
v_s3_input_bucket = "ds-mlops-s3" # S3 bucket for input and output data
v_s3_config_bucket="ds-mlops-s3"

###---------------

v_preprocessing_instance_type = "ml.m5.xlarge" # Instance type for preprocessing container it changes as per workload
v_prefix_for_drift_lambda="transformed/monitoring/inbound/xgb/code/evaluate_monitor_datadrift.zip" # drift to be changed
v_prefix_for_prep_lambda="transformed/monitoring/inbound/xgb/code/Monotoring_data_Prep_Preprocess.zip"
v_prefix_for_evaluate_lambda="transformed/monitoring/inbound/xgb/code/EvaluateMonitor_Payload.zip"
v_monitor_work_area="transformed/monitoring/monitor/xgb/curschedule/"


jsonlpath = ("s3://{}/{}/{}".format(v_s3_input_bucket,"transformed/monitoring/inbound/xgb",timestr))
statspath = "s3://{}/{}".format(v_s3_config_bucket,"transformed/monitoring/inbound/json-files/statistics.json")
constraintspath = "s3://{}/{}".format(v_s3_config_bucket,"transformed/monitoring/inbound/json-files/constraints.json")
violationpath = "s3://{}/{}/{}".format(v_s3_input_bucket,"transformed/monitoring/outbound/datadrift/xgb",timestr)


v_prefix_for_score_output = "transformed/monitoring/inbound/xgb/currentrun/"  # Prefix where data is stored
v_prefix_for_code_location = "transformed/monitoring/inbound/xgb/code/score_xgb_processing_script.py" # prefix where code is stored
v_prefix_for_post_code_loc = "transformed/monitoring/inbound/xgb/code/score_xgb_post_processing_script.py" # prefix where code is stored
v_score_instance_type = "ml.m5.xlarge" # Instance type for training
v_validation_scoring_instance_type = "ml.m5.large" # Instance type for batch scoring
#added for glue job-

v_prefix_for_glue_job="code/Glue_bookmarkScript.py"
v_prefix_for_source="transformed/monitoring/outbound/xgb/scoreoutput/xgb/"

v_prefix_for_dest="transformed/monitoring/inbound/xgb/currentrun/"
v_glue_source="s3://{}/{}".format(v_s3_input_bucket,v_prefix_for_source)
v_glue_dest="s3://{}/{}".format(v_s3_input_bucket,v_prefix_for_dest)
glue_script_location="s3://{}/{}".format(v_s3_input_bucket,v_prefix_for_glue_job)

job_name="Glue_S3_Bookmark"
# job_name="glue_mlops_job"



# v_model_name = "ds-mlops-linear-learner-02" # Name of DS_MLOPS model to be kept
v_model_name = "ds-mlops-xgboost" # Name of DS_MLOPS model to be kept
#v_scoe = "s3://ds-mlops-s3/transformed/monitoring/outbound/ll/scoreoutput/jsonline/2022/03/28/08/jsonl"

#jobname = monitorjobname
Environment={
        'baseline_constraints': '/opt/ml/processing/baseline/constraints/constraints.json',
        'baseline_statistics': '/opt/ml/processing/baseline/stats/statistics.json',
        'dataset_format': '{"sagemakerCaptureJson":{"captureIndexNames":["endpointInput","endpointOutput"]}}',
        'dataset_source': '/opt/ml/processing/input/endpoint',
        'start_time':    ep_strttime,
        'end_time': ep_endtime,
        'metric_time': ep_strttime,
        'output_path': '/opt/ml/processing/output',
#         'post_analytics_processor_script': '/opt/ml/processing/code/postprocessing/postprocessor.py',
        'publish_cloudwatch_metrics': 'Disabled',
        'sagemaker_endpoint_name': 'poc',
        'sagemaker_monitoring_schedule_name': 'poc'
        
    }
# # VV added after design review
# ENable this for the VW execution
# sec_groups = ["sg-01d629a900f9b4d92"]
# subnets = ["subnet-07bd1dfe6aee76227",
#            "subnet-076950ecc89d4340b",
#            "subnet-0c5a462cb45a14bab"]

print("jsonpath:", jsonlpath)
print("statistics path:", statspath)
print("constraints path:", constraintspath)
print("violations path:", violationpath)
print("v_prefix_for_source:", v_prefix_for_source)

jsonpath: s3://ds-mlops-s3/transformed/monitoring/inbound/xgb/2022/04/04/21
statistics path: s3://ds-mlops-s3/transformed/monitoring/inbound/json-files/statistics.json
constraints path: s3://ds-mlops-s3/transformed/monitoring/inbound/json-files/constraints.json
violations path: s3://ds-mlops-s3/transformed/monitoring/outbound/datadrift/xgb/2022/04/04/21
v_prefix_for_source: transformed/monitoring/outbound/xgb/scoreoutput/xgb/


In [854]:
##VV updated after review
execution_input = ExecutionInput(
    schema={
        "GlueJobName":str,
        "Post-processing": str,
        "MonitorJobName": str,
        "EvaluateDrift" : str
               }
)

In [ ]:
#%$$$% Add the timestamps partitioning for the currentruns...

# Step 1- Glue job to consolidate CSV out files 

In [807]:
%%writefile Glue_bookmarkScript.py
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

args = getResolvedOptions(sys.argv, ["JOB_NAME","S3_SOURCE", "S3_DEST"])
sc = SparkContext()
glueContext = GlueContext(sc)
glueContext.purge_s3_path(args["S3_DEST"], options={"retentionPeriod":0})
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args["JOB_NAME"], args)

# Script generated for node S3 bucket
S3bucket_node1 = glueContext.create_dynamic_frame.from_options(
    format_options={"quoteChar": '"', "withHeader": False, "separator": ","},
    connection_type="s3",
    format="csv",
    connection_options={
        "paths": [args["S3_SOURCE"]],
        "recurse": True,
    },
    transformation_ctx="S3bucket_node1",
)

# # Script generated for node Apply Mapping
# ApplyMapping_node1648633086293 = ApplyMapping.apply(
#     frame=S3bucket_node1,
#     mappings=[
#         ("col0", "long", "col0", "long"),
#         ("col1", "string", "col1", "string"),
#         ("col2", "choice", "col2", "choice"),
#         ("col3", "choice", "col3", "choice"),
#         ("col4", "choice", "col4", "choice"),
#         ("col5", "string", "col5", "string"),
#         ("col6", "choice", "col6", "choice"),
#     ],
#     transformation_ctx="ApplyMapping_node1648633086293",
# )

# Script generated for node S3 bucket
S3bucket_node3 = glueContext.getSink(
    path=args["S3_DEST"],
    connection_type="s3",
    updateBehavior="UPDATE_IN_DATABASE",
    partitionKeys=[],
    enableUpdateCatalog=True,
    transformation_ctx="S3bucket_node3",
)
S3bucket_node3.setCatalogInfo(
    catalogDatabase="incremental_s3_Volkswagen", catalogTableName="GLUE_ETL"
)
S3bucket_node3.setFormat("csv")
S3bucket_node3.writeFrame(S3bucket_node1)
job.commit()


Overwriting Glue_bookmarkScript.py


In [808]:
#push local to required S3 location
! aws s3 cp Glue_bookmarkScript.py s3://$v_s3_input_bucket/$v_prefix_for_glue_job
print(glue_script_location)

upload: ./Glue_bookmarkScript.py to s3://ds-mlops-s3/code/Glue_bookmarkScript.py
s3://ds-mlops-s3/code/Glue_bookmarkScript.py


In [809]:
# glue_client = boto3.client("glue")
# response = glue_client.create_job(
#     Name=job_name,
#     Description="PySpark job to extract the data and split in to training and validation data sets",
#     Role=v_glue_role,  # you can pass your existing AWS Glue role here if you have used Glue before
#     ExecutionProperty={"MaxConcurrentRuns": 2},
#     Command={"Name": "gluebookrmaretl", "ScriptLocation": glue_script_location, "PythonVersion": "3"},
#     DefaultArguments={"--job-language": "Python 3","--job-bookmark-option":"job-bookmark-disable"},
#     GlueVersion="3.0",
#     WorkerType="G.2X",
#     NumberOfWorkers=2,
#     Timeout=60,
# )

In [810]:
#define glue step for data consolidation/movement
monitor_csv_step = steps.GlueStartJobRunStep(
    "CSV Consolidation",
    parameters={
        "JobName": execution_input["GlueJobName"],
        "Arguments": {
            "--S3_SOURCE": v_glue_source,# scoring output location
            "--S3_DEST":v_glue_dest # monitoring work area
         },
    },
)

In [ ]:
# The folder path (inbound and outbound paths) to be corrected. Jsonline and currentrun...

##  All gluejobs should have a specific logical name(S).

In [ ]:
# Check to end this job if there is data or not. That should be in this step. 

In [ ]:
# We could bring in the post-processing job here into this glue job that takes care of all the data proocessing and we move to the monitoring piece directly. STEP 2

In [ ]:
# Why we need to do the scheduling? 

# Step 2- Evaluate presence of Data files

In [811]:
%%writefile EvaluateMonitor_Payload.py
import boto3
import logging
import json
import glob
logger = logging.getLogger()
logger.setLevel(logging.INFO)
client = boto3.client('s3') 
bucket="ds-mlops-s3"
prefix="transformed/monitoring/inbound/xgb/currentrun/"
def lambda_handler(event, context):
    resp=client.list_objects(
        Bucket=bucket,
        Prefix=prefix)
    if "Contents" in resp:
        return {'flag':1}
    else:
        return {'flag':0}
    

Overwriting EvaluateMonitor_Payload.py


In [812]:
#push function code to the local file
! zip EvaluateMonitor_Payload.zip EvaluateMonitor_Payload.py
#push local to required S3 location
! aws s3 cp EvaluateMonitor_Payload.zip s3://$v_s3_input_bucket/$v_prefix_for_evaluate_lambda

updating: EvaluateMonitor_Payload.py (deflated 40%)
upload: ./EvaluateMonitor_Payload.zip to s3://ds-mlops-s3/transformed/monitoring/inbound/xgb/code/EvaluateMonitor_Payload.zip


In [813]:
v_prefix_for_evaluate_lambda

'transformed/monitoring/inbound/xgb/code/EvaluateMonitor_Payload.zip'

In [814]:
##give lambda function name as appropriate, give path of S3 where you have uploaded the code, give proper description
function_name = 'LinearLearnerMonitor-EvaluatePayload'
lambda_client = boto3.client('lambda')
try:
    chkexisting=lambda_client.get_function(FunctionName=function_name)
    responde=lambda_client.update_function_code(FunctionName=function_name,
        S3Bucket=v_s3_input_bucket,
        S3Key=v_prefix_for_evaluate_lambda)
except Exception as e:
    print(e)
    response = lambda_client.create_function(
        FunctionName=function_name,
        Runtime='python3.7',
        Role=v_lambda_execution_role,
        Handler='EvaluateMonitor_Payload.lambda_handler',
        Code={
            'S3Bucket':v_s3_input_bucket,
            'S3Key': '{}'.format(v_prefix_for_evaluate_lambda)
        },
        Description='prepare the files as per the last schedule and schedule freq',
        Timeout=15,
        MemorySize=128
    )


In [815]:
Evaluate_step = compute.LambdaStep(
    'Evaluate Payload',
    parameters={  
        "FunctionName": function_name
    }
)

In [816]:
decide_run_step = steps.states.Choice(
    'Monitor Payload Present'
)

In [817]:
no_op_step = steps.states.Pass(
    'Bypass-No data file to process',
    comment='No Data file to process'
)

# Step 2- Sagemaker Job-consolidated CSV to JSONL conversion



In [818]:
%%writefile score_xgb_post_processing_script.py
import pandas as pd
import json
import glob
#path = r'/opt/ml/processing/input' # Input path
path='/opt/ml/processing/input/data'
all_files = glob.glob(path + "/*",recursive=True)
# all_files=['s3://ds-mlops-s3/data/scoreoutput/lr/2022/02/17/18/batchscoring.csv.out']
counter = 0
print(all_files)
for filename in all_files:
    print("hi")
    df = pd.read_csv(filename,header=None)
    df = df.sample(frac=.25)
    df=df.iloc[:,1:]
    # Create a multiline json
    json_list = json.loads(df.to_json(orient = "records"))
    output_path = "/opt/ml/processing/xgb/datacapture.jsonl" #path to the linear learner
    print(output_path)
    counter = counter + 1
    data = {}
    data["captureData"]={
            "endpointInput": {
                "observedContentType": "text/csv",
                "mode": "INPUT",
                "data": "132,25,113.2,96,269.9,107,229.1,87,7.1,7,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1",
                "encoding": "CSV"
            },
            "endpointOutput": {
                "observedContentType": "text/csv; charset=utf-8",
                "mode": "OUTPUT",
                "data": "6295.23876953125",
                "encoding": "CSV"
            }
        }
    data["eventMetadata"] = {
            "eventId": "",
            "inferenceTime": "2"
        }
    data["eventVersion"] = "0"
    with open(output_path, 'w') as f:
        print(output_path)
        for item in json_list:
            item = list(item.values())
            inpitem = ','.join([str(elem) for elem in item[:-1]])
            #outitem = ','.join([str(elem) for elem in item[-1]])
            outitem=str(item[-1])
            data["captureData"]["endpointInput"]["data"] = inpitem
            data["captureData"]["endpointOutput"]["data"] =outitem
            f.write("%s\n" % data)
# Data push




Overwriting score_xgb_post_processing_script.py


In [819]:
!aws s3 cp score_xgb_post_processing_script.py s3://$v_s3_input_bucket/$v_prefix_for_post_code_loc # Uploading preprocessing code on s3

upload: ./score_xgb_post_processing_script.py to s3://ds-mlops-s3/transformed/monitoring/inbound/xgb/code/score_xgb_post_processing_script.py


In [820]:
processor = Processor(image_uri='683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.20.0-cpu-py3',
                     role=v_preprocessing_iam_role,
                     instance_count=1,
                     instance_type=v_preprocessing_instance_type
                     #network_config = NetworkConfig(security_group_ids = sec_groups, subnets = subnets)
                     )

In [821]:
input_data = "s3://{}/{}".format(v_s3_input_bucket,v_prefix_for_score_output) #updated on 30 March
input_code = "s3://{}/{}".format(v_s3_input_bucket,v_prefix_for_post_code_loc)
output_data = jsonlpath

inputs = [
    ProcessingInput(
        source=input_data, destination="/opt/ml/processing/input/data", input_name="input"
    ),
    ProcessingInput(
        source=input_code,
        destination="/opt/ml/processing/input/code",
        input_name="code",
    ),
]

outputs = [
    ProcessingOutput(
        source="/opt/ml/processing/xgb",
        destination="{}/{}".format(output_data, ""),
        output_name="xgb_data",
    )
    
]

In [822]:
print(input_data)

s3://ds-mlops-s3/transformed/monitoring/inbound/xgb/currentrun/


In [823]:
postprocessing_step = ProcessingStep(
    state_id='Post-processing', 
    processor=processor,
    job_name=execution_input["Post-processing"], 
    inputs=inputs, 
    outputs=outputs, 
    experiment_config=None, 
    container_entrypoint=["python3", "/opt/ml/processing/input/code/score_xgb_post_processing_script.py"], # DS needs to change this directory /path
    wait_for_completion=True
)

In [824]:
# Generate unique names for Pre-Processing Job, Training Job, and Model Evaluation Job for the Step Functions Workflow
 # Each Training Job requires a unique name
batch_scoring_job_name = "xgb-post-preprocessing-{}".format(
    uuid.uuid1().hex
)

# Step 3: Sagemaker Job-Execute Monitor Step

In [825]:
import boto3
client = boto3.client('sagemaker')


In [826]:
# Dynamic Job Name
import uuid
monitorjobname = "Batch-model-monitor-{}".format(
    uuid.uuid1().hex
)

In [827]:
# s3://ds-mlops-s3/data/scoreoutput/xgb/2022/03/21/12/test/outputfile_.json

Inputs=[
    ProcessingInput(
        source=jsonlpath, 
        destination=("{}/{})".format("/opt/ml/processing/input/endpoint/poc/AllTraffic",timestr)),
        input_name="input_1"
    ),
    ProcessingInput(
        source=statspath,
        destination="/opt/ml/processing/baseline/stats",
        input_name="baseline"
    ),
    ProcessingInput(
        source=constraintspath,
        destination="/opt/ml/processing/baseline/constraints",
        input_name="constraints",
    )
 ]
Output= [
       ProcessingOutput(
        source="/opt/ml/processing/output",
        destination=violationpath,
        output_name="result",
    )    
]


In [828]:
processor = Processor(image_uri='156813124566.dkr.ecr.us-east-1.amazonaws.com/sagemaker-model-monitor-analyzer',
                     role=v_preprocessing_iam_role,
                     instance_count=1,
                     instance_type=v_preprocessing_instance_type,
                     env=Environment,
                     max_runtime_in_seconds=600)
                     #network_config = NetworkConfig(security_group_ids = sec_groups, subnets = subnets))

In [829]:
monitoring_step = ProcessingStep(
    state_id='Batch Monitoring', 
    processor=processor,
    job_name=execution_input["MonitorJobName"], 
    inputs=Inputs, 
    outputs=Output,
    experiment_config=None,
    wait_for_completion=True
)

# Step 4: Lambda- Evaluate Drift and send notification

In [830]:
# Understanding of the drift patterns:

# 1. Actual values vs the predicted values
# 2. The baselines metric on the stats vs constraint files. 
# 3. An evaluation on the baselines to detect the drift patterns based on the comparison between the actual vs
#     predicted values

In [831]:
violations_filepath = ("{}/{}/{}".format("transformed/monitoring/outbound/datadrift/xgb",timestr,"constraint_violations.json"))

print(violations_filepath)



transformed/monitoring/outbound/datadrift/xgb/2022/04/04/20/constraint_violations.json


In [832]:
%%writefile evaluate_monitor_drift.py


import random
import csv
import json
import boto3
import json
import logging
import datetime
year=datetime.datetime.now().strftime("%Y")
month=datetime.datetime.now().strftime("%m")
day=datetime.datetime.now().strftime("%d")
hour=datetime.datetime.now().strftime("%H")
ehour=datetime.datetime(int(year),int(month),int(day),int(hour)+1).strftime("%H")
timestr=("{}/{}/{}/{}".format(year,month,day,hour))
#timestr=("{}/{}/{}/{}".format("2022","03","30","10"))

logger = logging.getLogger()
logger.setLevel(logging.INFO)
sm_client = boto3.client('sagemaker')


v_s3_input_bucket = "ds-mlops-s3"
violations_filepath = ("{}/{}/{}".format("transformed/monitoring/outbound/datadrift/xgb",timestr,"constraint_violations.json"))
print(violations_filepath)

def lambda_handler(event, context):
    try:
        s3_client = boto3.client('s3')
        response = s3_client.get_object(Bucket=v_s3_input_bucket,Key=violations_filepath)
        text = response['Body'].read().decode('utf-8')
        data = json.loads(text)
        print(data)
        subject = "Data Drift Monitor - Xgboost Learner"
        body = data
        sns_topic_arn="arn:aws:sns:us-east-1:014257795134:topic-lambda-mlops"
        sns_client = boto3.client('sns')
        response = sns_client.publish(
        TargetArn=sns_topic_arn,
        Message=json.dumps({'default': json.dumps(f"{data}"),
                            'sms': subject,
                            'email': body}),
        Subject=subject,
        MessageStructure='json')
        return response

    except Exception as e:
        return "No Drift detected"
        


Overwriting evaluate_monitor_drift.py


In [833]:
# s3://ds-mlops-s3/transformed/monitoring/outbound/datadrift/ll/

In [834]:
#push function code to the local file
! zip evaluate_monitor_drift.zip evaluate_monitor_drift.py
#push local to required S3 location
! aws s3 cp evaluate_monitor_drift.zip s3://$v_s3_input_bucket/$v_prefix_for_drift_lambda

updating: evaluate_monitor_drift.py (deflated 57%)
upload: ./evaluate_monitor_drift.zip to s3://ds-mlops-s3/transformed/monitoring/inbound/xgb/code/evaluate_monitor_datadrift.zip


In [836]:
##give lambda function name as appropriate, give path of S3 where you have uploaded the code, give proper description
function_name = 'LinearLearner_Monitor-drift'
lambda_client = boto3.client('lambda')
try:
    chkexisting=lambda_client.get_function(FunctionName=function_name)
    responde=lambda_client.update_function_code(FunctionName=function_name,
        S3Bucket=v_s3_input_bucket,
        S3Key=v_prefix_for_drift_lambda)
except Exception as e:
    print(e)
    response = lambda_client.create_function(
        FunctionName=function_name,
        Runtime='python3.7',
        Role=v_lambda_execution_role,
        Handler='evaluate_monitor_drift.lambda_handler',
        Code={
            'S3Bucket':v_s3_input_bucket,
            'S3Key': '{}'.format(v_prefix_for_drift_lambda)
        },
        Description='notify the data or model drift as per violation file data points',
        Timeout=15,
        MemorySize=128
    )


In [837]:
##define Lambda Step as below and add it in the overall execution flow of the step machine
lambda_step = compute.LambdaStep(
    'Evaluate Drift',
    parameters={  
        "FunctionName": function_name
    }
)

In [838]:
threshold_rule = steps.choice_rule.ChoiceRule.NumericEquals(variable=Evaluate_step.output()['Payload']['flag'], value=1)
decide_run_step.add_choice(rule=threshold_rule, next_step=postprocessing_step)
decide_run_step.default_choice(next_step=no_op_step)
postprocessing_step.next(monitoring_step)
monitoring_step.next(lambda_step)

Evaluate Drift LambdaStep(parameters={'FunctionName': 'LinearLearner_Monitor-drift'}, resource='arn:aws:states:::lambda:invoke', type='Task')

In [839]:

# basic_path3=Chain([postprocessing_step])

basic_path=Chain([monitor_csv_step,Evaluate_step,decide_run_step])


In [840]:
# Next, we define the workflow
basic_workflowv1 = Workflow(
    name="ds-mlops-monitor-step-functionv1",
    definition=basic_path,
    role=v_workflow_execution_role
)

#Render the workflow
basic_workflowv1.render_graph()

In [841]:
print(basic_workflowv1.definition.to_json(pretty=True)) # From this json we would be leveraging the codes to create the Cloud Formation parameterized template...

{
    "StartAt": "CSV Consolidation",
    "States": {
        "CSV Consolidation": {
            "Parameters": {
                "JobName.$": "$$.Execution.Input['GlueJobName']",
                "Arguments": {
                    "--S3_SOURCE": "s3://ds-mlops-s3/transformed/monitoring/outbound/xgb/scoreoutput/xgb/",
                    "--S3_DEST": "s3://ds-mlops-s3/transformed/monitoring/inbound/xgb/currentrun/"
                }
            },
            "Resource": "arn:aws:states:::glue:startJobRun.sync",
            "Type": "Task",
            "Next": "Evaluate Payload"
        },
        "Evaluate Payload": {
            "Parameters": {
                "FunctionName": "LinearLearnerMonitor-EvaluatePayload"
            },
            "Resource": "arn:aws:states:::lambda:invoke",
            "Type": "Task",
            "Next": "Monitor Payload Present"
        },
        "Monitor Payload Present": {
            "Type": "Choice",
            "Choices": [
                {
         

In [842]:
basic_workflowv1.create()

[ERROR] A workflow with the same name already exists on AWS Step Functions. To update a workflow, use Workflow.update().


'arn:aws:states:us-east-1:014257795134:stateMachine:ds-mlops-monitor-step-functionv1'

In [843]:
basic_workflowv1.update(definition=basic_workflowv1.definition,role=basic_workflowv1.role)

[INFO] Workflow updated successfully on AWS Step Functions. All execute() calls will use the updated definition and role within a few seconds. 


'arn:aws:states:us-east-1:014257795134:stateMachine:ds-mlops-monitor-step-functionv1'

In [844]:
basic_workflow_execution = basic_workflowv1.execute(
    inputs={
       "MonitorJobName": monitorjobname,
       "Post-processing": batch_scoring_job_name,
        "GlueJobName": job_name
            }
)

[INFO] Workflow execution started successfully on AWS Step Functions.


In [845]:
basic_workflow_execution.render_progress()